In [2]:
%load_ext nb_black

import numpy

<IPython.core.display.Javascript object>

In [3]:
1, 1

(1, 1)

<IPython.core.display.Javascript object>

In [ ]:
import subprocess



def wccount(filename):
    out = subprocess.run(['wc', '-l', filename],
                         stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT
                         ).stdout
    return int(out.partition(b' ')[0])



glove_file = "data/glove.6B.300d.txt"
embeddings_dict = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in tqdm(f, "Parsing glove embeddings", total=wccount(glove_file), ncols=100):
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

embeds = pd.DataFrame(embeddings_dict)
emb_np = embeds.to_numpy().T

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

def translate(words):
    batch = tokenizer(words, return_tensors="pt", padding=True)
    gen = model.generate(**batch)
    return tokenizer.batch_decode(gen, skip_special_tokens=True)

In [ ]:
from collections import Counter
from scipy import spatial

kdtree = spatial.KDTree(emb_np)

In [ ]:
english_topics = pd.DataFrame()
for col in tqdm(topics, "Translating topic words to english", total=len(topics.columns), position=0, ncols=100):
    words = translate(list(topics[col]))
    words = [''.join([c.lower() for c in word.split()[0] if c.isalpha()]) for word in words]
    english_topics[col] = np.array(words)

In [ ]:
for col, words in tqdm(topics.iteritems(),
                       "Finding names for topics", total=topics.shape[1], position=0, ncols=100):
    words = translate(list(words))
    words = [''.join([c.lower() for c in word.split()[0] if c.isalpha()]) for word in words]
    counter = Counter()
    for word in words:
        if word not in embeds:
            continue
        point = embeds[word].to_numpy()
        distance, indices = kdtree.query(point, range(1, 10))
        for index in indices:
            counter[embeds.columns[index]] += 1
            
    topics.rename({col: counter.most_common()[0][0]}, axis='columns', inplace=True)

In [1]:
import parsl
from parsl.app.app import python_app, bash_app



parsl.load()

In [2]:
@python_app
def slow_hello():
    import time
    time.sleep(5)
    return 'Hello World!'

In [7]:
sh = slow_hello()

In [19]:
print(sh.result() if sh.done() else 'Wait')

Hello World!
